In [1]:
# Run a mock of fractal-demos/examples/01, with an additional
# illumination correction task which does *not* overwrite input
# images.

In [2]:
# Preliminary imports and auxiliary-function definition

from pathlib import Path
from devtools import debug
import tempfile

from fractal_server.app.runner.v2.models import Dataset, WorkflowTask
from fractal_server.app.runner.v2.runner import execute_tasks_v2
from fractal_server.images import SingleImage

import sys
sys.path.append("../tests/v2/04_runner")
from fractal_tasks_core_mock import TASK_LIST

from concurrent.futures import ThreadPoolExecutor

executor = ThreadPoolExecutor()



def image_data_exist_on_disk(image_list: list[SingleImage]):
    """
    Given an image list, check whether mock data were written to disk.
    """
    all_images_have_data = True
    for image in image_list:
        if (Path(image.path) / "data").exists():
            print(f"[image_data_exist_on_disk] {image.path} contains data")
        else:
            print(f"[image_data_exist_on_disk] {image.path} does *not* contain data")
            all_images_have_data = False
        
    return all_images_have_data

{'name': 'create_ome_zarr_compound', 'input_flags': {}, 'output_flags': {}, 'function_non_parallel': <cyfunction create_ome_zarr at 0x71afc042ab50>, 'function_parallel': <cyfunction yokogawa_to_zarr at 0x71afc042be00>}
{'name': 'create_ome_zarr_multiplex_compound', 'input_flags': {}, 'output_flags': {}, 'function_non_parallel': <cyfunction create_ome_zarr_multiplex at 0x71afc042b440>, 'function_parallel': <cyfunction yokogawa_to_zarr at 0x71afc042be00>}
{'name': 'MIP_compound', 'input_flags': {'has_z': True}, 'output_flags': {'has_z': False}, 'function_non_parallel': <cyfunction new_ome_zarr at 0x71afc046a0c0>, 'function_parallel': <cyfunction maximum_intensity_projection at 0x71afc046aa80>}
{'name': 'illumination_correction', 'input_flags': {'illumination_correction': False}, 'output_flags': {'illumination_correction': True}, 'function_non_parallel': None, 'function_parallel': <cyfunction illumination_correction at 0x71afc04686c0>}
{'name': 'illumination_correction_compound', 'input_f

In [3]:
# Create temporary directory for mocked zarrs
tmp_path = Path(tempfile.mkdtemp())
zarr_dir = (tmp_path / "zarr_dir").as_posix().rstrip("/")
print(f"{zarr_dir=}")

zarr_dir='/tmp/tmp6oomqoz4/zarr_dir'


In [4]:
# Run create-ome-zarr (starting from empty Dataset)
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["create_ome_zarr_compound"],
            args_non_parallel=dict(image_dir="/tmp/input_images"),
        )
    ],
    dataset=Dataset(zarr_dir=zarr_dir),
    executor=executor,
)

# Images were added, but data do not exist on-disk yet
assert image_data_exist_on_disk(dataset.images)

# Print dataset metadata
debug(dataset)

[create_ome_zarr] START
[create_ome_zarr] image_dir='/tmp/input_images'
[create_ome_zarr] zarr_dir='/tmp/tmp6oomqoz4/zarr_dir'
[create_ome_zarr] zarr_path='/tmp/tmp6oomqoz4/zarr_dir/my_plate.zarr'
[create_ome_zarr] END
[yokogawa_to_zarr] START
[yokogawa_to_zarr] path='/tmp/tmp6oomqoz4/zarr_dir/my_plate.zarr/A/01/0'
[yokogawa_to_zarr] raw_path='/tmp/input_images/A_01_0.tif'
[yokogawa_to_zarr] START
[yokogawa_to_zarr] path='/tmp/tmp6oomqoz4/zarr_dir/my_plate.zarr/A/02/0'
[yokogawa_to_zarr] raw_path='/tmp/input_images/A_02_0.tif'
[yokogawa_to_zarr] END
[yokogawa_to_zarr] END
[image_data_exist_on_disk] /tmp/tmp6oomqoz4/zarr_dir/my_plate.zarr/A/01/0 contains data
[image_data_exist_on_disk] /tmp/tmp6oomqoz4/zarr_dir/my_plate.zarr/A/02/0 contains data
/tmp/ipykernel_57249/201959533.py:17 <module>
    dataset: Dataset(
        id=None,
        history=[
            'create_ome_zarr_compound',
        ],
        zarr_dir='/tmp/tmp6oomqoz4/zarr_dir',
        images=[
            SingleImage(
   

Dataset(id=None, history=['create_ome_zarr_compound'], zarr_dir='/tmp/tmp6oomqoz4/zarr_dir', images=[SingleImage(path='/tmp/tmp6oomqoz4/zarr_dir/my_plate.zarr/A/01/0', origin=None, attributes={'well': 'A01', 'plate': 'my_plate.zarr'}, flags={'has_z': True}), SingleImage(path='/tmp/tmp6oomqoz4/zarr_dir/my_plate.zarr/A/02/0', origin=None, attributes={'well': 'A02', 'plate': 'my_plate.zarr'}, flags={'has_z': True})], attribute_filters={'plate': 'my_plate.zarr'}, flag_filters={'has_z': True})

[illumination_correction] START[illumination_correction] START
[illumination_correction] path='/tmp/tmp6oomqoz4/zarr_dir/my_plate.zarr/A/02/0'
[illumination_correction] overwrite_input=False

[illumination_correction] path='/tmp/tmp6oomqoz4/zarr_dir/my_plate.zarr/A/01/0'
[illumination_correction] overwrite_input=False
[illumination_correction] new_path='/tmp/tmp6oomqoz4/zarr_dir/my_plate.zarr/A/02/0_corr'
[illumination_correction] new_path='/tmp/tmp6oomqoz4/zarr_dir/my_plate.zarr/A/01/0_corr'
[illumination_correction] END
[illumination_correction] END
[new_ome_zarr] START
[new_ome_zarr] paths=['/tmp/tmp6oomqoz4/zarr_dir/my_plate.zarr/A/01/0_corr', '/tmp/tmp6oomqoz4/zarr_dir/my_plate.zarr/A/02/0_corr']
[new_ome_zarr] Identified old_plate='my_plate.zarr'
[new_ome_zarr] new_plate='my_plate_mip.zarr'
[new_ome_zarr] new_zarr_path='/tmp/tmp6oomqoz4/zarr_dir/my_plate_mip.zarr'
[new_ome_zarr] END
[maximum_intensity_projection] START
[maximum_intensity_projection] path='/tmp/tmp6oomqoz4/zarr_di

In [5]:
# Run illumination correction, with overwrite_input=True
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["illumination_correction"],
            args_parallel=dict(overwrite_input=False),
        )
    ],
    dataset=dataset,
    executor=executor,
)

# Image data still exist (and they actually were modified)
assert image_data_exist_on_disk(dataset.images)

# Print dataset metadata
debug(dataset);

[image_data_exist_on_disk] /tmp/tmp6oomqoz4/zarr_dir/my_plate.zarr/A/01/0 contains data
[image_data_exist_on_disk] /tmp/tmp6oomqoz4/zarr_dir/my_plate.zarr/A/02/0 contains data
[image_data_exist_on_disk] /tmp/tmp6oomqoz4/zarr_dir/my_plate.zarr/A/01/0_corr contains data
[image_data_exist_on_disk] /tmp/tmp6oomqoz4/zarr_dir/my_plate.zarr/A/02/0_corr contains data
/tmp/ipykernel_57249/3107743349.py:17 <module>
    dataset: Dataset(
        id=None,
        history=[
            'create_ome_zarr_compound',
            'illumination_correction',
        ],
        zarr_dir='/tmp/tmp6oomqoz4/zarr_dir',
        images=[
            SingleImage(
                path='/tmp/tmp6oomqoz4/zarr_dir/my_plate.zarr/A/01/0',
                origin=None,
                attributes={
                    'well': 'A01',
                    'plate': 'my_plate.zarr',
                },
                flags={
                    'has_z': True,
                },
            ),
            SingleImage(
         

In [6]:
# Run new-ome-zarr, as an init task for MIP
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["MIP_compound"],
            args_non_parallel=dict(suffix="mip"),
        )
    ],
    dataset=dataset,
    executor=executor,
)

# 3D image data exist on disk
images_3D = [
        image
        for image in dataset.images
        if image.flags["has_z"]
    ]
assert image_data_exist_on_disk(images_3D)

# 2D image data do not exist on disk
images_2D = [
        image
        for image in dataset.images
        if not image.flags["has_z"]
    ]
assert image_data_exist_on_disk(images_2D)

# Print dataset metadata
debug(dataset);

[image_data_exist_on_disk] /tmp/tmp6oomqoz4/zarr_dir/my_plate.zarr/A/01/0 contains data
[image_data_exist_on_disk] /tmp/tmp6oomqoz4/zarr_dir/my_plate.zarr/A/02/0 contains data
[image_data_exist_on_disk] /tmp/tmp6oomqoz4/zarr_dir/my_plate.zarr/A/01/0_corr contains data
[image_data_exist_on_disk] /tmp/tmp6oomqoz4/zarr_dir/my_plate.zarr/A/02/0_corr contains data
[image_data_exist_on_disk] /tmp/tmp6oomqoz4/zarr_dir/my_plate_mip.zarr/A/01/0_corr contains data
[image_data_exist_on_disk] /tmp/tmp6oomqoz4/zarr_dir/my_plate_mip.zarr/A/02/0_corr contains data
/tmp/ipykernel_57249/1847195232.py:30 <module>
    dataset: Dataset(
        id=None,
        history=[
            'create_ome_zarr_compound',
            'illumination_correction',
            'MIP_compound',
        ],
        zarr_dir='/tmp/tmp6oomqoz4/zarr_dir',
        images=[
            SingleImage(
                path='/tmp/tmp6oomqoz4/zarr_dir/my_plate.zarr/A/01/0',
                origin=None,
                attributes={
     

In [7]:
# Run cellpose on current images
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["cellpose_segmentation"],
            args_parallel=dict(),
        )
    ],
    dataset=dataset,
    executor=executor,
)

# All image data (3D and 2D) now exists on disk
assert image_data_exist_on_disk(dataset.images)

# Print dataset metadata
debug(dataset);

[image_data_exist_on_disk] /tmp/tmp6oomqoz4/zarr_dir/my_plate.zarr/A/01/0 contains data
[image_data_exist_on_disk] /tmp/tmp6oomqoz4/zarr_dir/my_plate.zarr/A/02/0 contains data
[image_data_exist_on_disk] /tmp/tmp6oomqoz4/zarr_dir/my_plate.zarr/A/01/0_corr contains data
[image_data_exist_on_disk] /tmp/tmp6oomqoz4/zarr_dir/my_plate.zarr/A/02/0_corr contains data
[image_data_exist_on_disk] /tmp/tmp6oomqoz4/zarr_dir/my_plate_mip.zarr/A/01/0_corr contains data
[image_data_exist_on_disk] /tmp/tmp6oomqoz4/zarr_dir/my_plate_mip.zarr/A/02/0_corr contains data
/tmp/ipykernel_57249/1519895496.py:17 <module>
    dataset: Dataset(
        id=None,
        history=[
            'create_ome_zarr_compound',
            'illumination_correction',
            'MIP_compound',
            'cellpose_segmentation',
        ],
        zarr_dir='/tmp/tmp6oomqoz4/zarr_dir',
        images=[
            SingleImage(
                path='/tmp/tmp6oomqoz4/zarr_dir/my_plate.zarr/A/01/0',
                origin=Non

In [8]:
executor.shutdown()